In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
data = pd.read_csv('data.csv')
test = pd.read_csv('test.csv')
predictions = pd.DataFrame({'id':test['id']})
predictions = predictions.set_index('id')


In [3]:
data.drop(['cut','depth','table','x','y','z'],axis=1,inplace=True)
test.drop(['id','cut','depth','table','x','y','z'],axis=1,inplace=True)

In [106]:
## visualize data
#display(data.head())
#print(data.shape)

In [107]:
## count missing values
#data.isnull().sum()

In [108]:
## check variable types
#data.dtypes

In [109]:
#print(data['cut'].unique())
#print(data['color'].unique())
#print(data['clarity'].unique())

In [110]:
#data.describe()

In [111]:
#data.plot.hist(bins=30)

In [112]:
#corrMatrix = data.corr()
#corrMatrix.style.background_gradient(cmap='coolwarm')

In [113]:
#print(data['color'].value_counts())
#print(data['clarity'].value_counts())

In [4]:
data_dummy = pd.get_dummies(data,columns = ['color','clarity'],drop_first=True)
test_dummy = pd.get_dummies(test,columns = ['color','clarity'],drop_first=True)
X = data_dummy.drop(['price'],axis=1)
y = data['price']

In [5]:
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.decomposition import SparsePCA


steps = [
    StandardScaler(),
    Normalizer()
]

pipe = make_pipeline(*steps)

X = pipe.fit_transform(X)
X_test_test = pipe.fit_transform(test_dummy)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [14]:
from sklearn import svm
svm_model = svm.SVR(gamma='auto')
svm_model.fit(X_train,y_train)

from sklearn import tree
tree_model = tree.DecisionTreeRegressor()
tree_model.fit(X_train,y_train)

from sklearn.linear_model import Lasso
lasso_model = Lasso()
lasso_model.fit(X_train,y_train)

import statsmodels.api as sm
ols_model = sm.OLS(y_train,X_train).fit()

from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()
linear_reg.fit(X_train,y_train)

from sklearn.ensemble import RandomForestRegressor
forest_model = RandomForestRegressor(n_estimators=200,max_depth=50)
forest_model.fit(X_train,y_train)

models = [svm_model,tree_model,lasso_model,ols_model,linear_reg,forest_model]

In [16]:
for i in models:
    y_pred = i.predict(X_test)
    print('---------------')
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    print(f'\n\n{i}: {rmse}')

---------------


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False): 4030.006519225601
---------------


DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best'): 656.639394105993
---------------


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False): 1446.622413014839
---------------


<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x11ff07d10>: 4205.38668876179
---------------


LinearRegression(copy_X=True, fit_intercept=True,

In [17]:
## elegimos el mejor modelo --> random forest

y_pred = forest_model.predict(X_test_test)
predictions['price'] = y_pred

In [18]:
predictions.to_csv('sample_submission.csv')